In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Load the datasets
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# One-hot encoding for 'Sex' 
ohe = OneHotEncoder()
sex_encoded = ohe.fit_transform(train_data[['Sex']]).toarray()
sex_columns = ohe.get_feature_names_out(['Sex'])
sex_df = pd.DataFrame(sex_encoded, columns=sex_columns)

# Merge encoded columns and drop original 'Sex' column
train_data = pd.concat([train_data, sex_df], axis=1)
train_data.drop('Sex', axis=1, inplace=True)

# Define features and target
X = train_data.drop(['Rings', 'id'], axis=1)
y = train_data['Rings']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=1234)

# Ridge Regression with cross-validation
alphas = np.logspace(3, 9, 13) #note, any number here less than 3 yields convergence errors.
ridge = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_log_error')
ridge.fit(X_train, y_train)

# Predictions
y_pred_ridge = ridge.predict(X_val)
y_pred_ridge = np.maximum(y_pred_ridge, 0)

# RMSLE training
rmsle_ridge = np.sqrt(mean_squared_log_error(y_val, y_pred_ridge))
print(f'RMSLE (Ridge Regression): {rmsle_ridge}')

RMSLE (Ridge Regression): 0.16233109018505515


In [31]:
#test set one-hot encoding
# One-hot encoding for 'Sex' 
ohe = OneHotEncoder()
sex_encoded = ohe.fit_transform(test_data[['Sex']]).toarray()
sex_columns = ohe.get_feature_names_out(['Sex'])
sex_df = pd.DataFrame(sex_encoded, columns=sex_columns)

# Merge encoded columns and drop original 'Sex' column
test_data = pd.concat([test_data, sex_df], axis=1)
test_data.drop('Sex', axis=1, inplace=True)

#keep IDs separate
test_ID = test_data['id']

#drop ID from dataset
test_data.drop('id', axis=1, inplace=True)

# Predict on the test set using the trained Ridge model
test_predictions = ridge.predict(test_data)
test_predictions = np.maximum(test_predictions, 0)  # Ensure predictions are >= 0

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RidgeCV was fitted without feature names
  warnings.warn(


array([10.29792026, 10.41293186, 10.498901  , ..., 10.35136372,
       10.97049415, 10.25478007])

In [36]:
# Prepare the submission file
#round the predictions to integers
submission = pd.DataFrame({
    'id': test_ID,  
    'Rings': test_predictions.round().astype(int)
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print('Submission file created: submission.csv')



submission


Submission file created: submission.csv


,id,Rings
0,90615,10
1,90616,10
2,90617,10
3,90618,11
4,90619,10
...,...,...
60406,151021,10
60407,151022,10
60408,151023,10
60409,151024,11
